# Devoir 4, Question 3 : Transfert de style

# Homework 4, Question 3: Style transfer

## Code préambule

## Preamble code

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import numpy
import os
import requests
import time
import pandas
pandas.set_option('display.max_colwidth', 0)

from IPython import display
import matplotlib.pyplot as plt
from io import BytesIO
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
import torchvision.models as models
from torchvision import transforms
from tqdm import tqdm

from PIL import Image

# Constantes / Constants
IMG_SIZE = 256
IMAGENET_MEAN = [0.485, 0.456, 0.406] # Moyenne pour chaque canal de couleur
IMAGENET_STD = [0.229, 0.224, 0.225]  # Std pour chaque canal de couleur
STYLE_IMAGE = 'style_image'
CONTENT_IMAGE = 'content_image'
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Variables
results = {'Name':[],
           'Shape':[],
           'Mean':[],
           'Std':[],
          }

def fetch_image(file_id):
    """
    Cette fonction télécharge une image que vous partagez de votre Google Drive.
    Elle retourne l'image dans un format PIL.
    This function downloads an image you share from your Google Drive.
    It returns the image in a PIL format.
    """
    URL = "https://drive.google.com/uc?"
    session = requests.Session()
    
    r = session.get(URL, params = { 'id' : file_id, 'alt' : 'media'}, stream = True)
    error_msg = f'ERROR: impossible to download the image (code={r.status_code})'
    assert(r.status_code == 200), error_msg
    
    params = { 'id' : file_id, 'confirm' : 'download_warning' }
    r = session.get(URL, params = params, stream = True)
    stream = BytesIO(r.content)
    image = Image.open(stream)
    return image
    
# Gram matrix
def gram_matrix(tensor):
    """
    Calcul de la matrice de Gram pour un tenseur donné
    Calculation of the Gram matrix for a given tensor 
    Gram Matrix: https://en.wikipedia.org/wiki/Gramian_matrix
    """
    
    # Get the (B, C, H, W) of the Tensor
    _, d, h, w = tensor.size()
    
    # Reshape tensor to multiply the features for each channel
    tensor = tensor.view(d, h * w)
    
    # Calculate the Gram matrix
    gram = torch.mm(tensor, tensor.t())
    
    return gram

def extract_features(image, model_features, layers=None):
    """
    Infère l'image dans le modèle et extrait les features pour
    les couches désirées. Les couches par défaut concordent
    avec celles du réseau VGG19 de Gatys et al. (2016).
    Infers the image into the model and extracts the features for
    the desired layers. The default layers are consistent with
    those of the VGG19 network of Gatys et al. (2016).
    """
    if layers is None:
        layers = {'0': 'conv1_1',
                  '2': 'conv1_2',
                  '5': 'conv2_1',
                  '7': 'conv2_2',
                  '10': 'conv3_1',
                  '12': 'conv3_2',
                  '19': 'conv4_1',
                  '21': 'conv4_2',
                  '28': 'conv5_1',
                  '30': 'conv5_2'}
                
    features = {}
    x = image
    for layer_idx, layer in enumerate(model_features):
        x = layer(x)
        if str(layer_idx) in layers:
            features[layers[str(layer_idx)]] = x
            
    return features

De manière générale, l'entraînement des réseaux de neurones en classement implique un entraînement où on observe la performance selon sa fonction de perte que l'on souhaite minimiser en validation, afin d'obtenir un modèle qui généralise bien. Le classement n'est pas le seul contexte d'apprentissage pour les réseaux de neurones, il existe plusieurs autres utilisations de réseaux de neurones, notamment pour permettre la génération d'images. Le contexte de génération d'images offre ainsi un retour visuel qui permet de donner une appréciation qualitative du fonctionnement du système. Si les images générées semblent réelles, on peut présumer que le modèle fonctionne bien! L'exercice suivant a alors été conçu pour vous permettre de mieux visualiser les performances, avec un retour visuel qui devrait être évocateur. 

À l'aide de PyTorch, vous allez mettre en application ce qu'on appelle le *transfert de style*, qui est un problème qui fait appel à la notion de transfert de représentation. À l'aide d'un réseau VGG19, on vous demande de suivre l'implémentation d'un article de recherche [Gatys et coll., 2016](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Gatys_Image_Style_Transfer_CVPR_2016_paper.pdf) afin de générer une image artistique. En utilisant un modèle fourni préentraîné sur ImageNet, nous voulons extraire le *style* d'une première image (à gauche) avec le réseau et l'appliquer sur le *contenu* d'une seconde image (à droite):

![Contenu et style, Mona-Lisa](https://pax.ulaval.ca/static/GIF-4101-7005/images/content_style_mona.png)

L'objectif est de créer une image hybride qui contient à la fois le style ainsi que le contenu de deux images. Il s'agit d'un exercice plus visuel qui vous permettra d'améliorer votre intuition sur le fonctionnement d'un réseau de neurones à convolution en vous basant sur l'information d'un article scientifique. Pour vous donner une idée du résultat, voici l'image hybride générée à partir des images présentées plus haut:

![Transfert Mona-Lisa](https://pax.ulaval.ca/static/GIF-4101-7005/images/style_transfer_mona.gif)

Le réseau de neurones à convolution VGG19 est composé de 2 groupes: les *features* et les couches de classification. Le style correspond au résultat du passage des filtres de la couche de *features* sur l'image d'entrée, alors que le contenu correspond aux valeurs des pixels de la deuxième image.

In general, training neural networks for classification involves training them to observe performance according to its loss function, which we wish to minimize in validation, in order to obtain a model that generalizes well. Classification is not the only training context for neural networks. They have several other uses, notably image generation. The context of image generation thus provides visual feedback that gives a qualitative assessment of the system's operation. If the generated images look real, we can assume that the model works well! The following exercise is designed to allow you to better visualize the performance, with visual feedback that should be evocative. 

Using PyTorch, you will implement what is called *style transfer*, which is a problem that involves the notion of representation transfer. Using a VGG19 network, you are asked to follow the implementation of a research paper [Gatys et al., 2016](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Gatys_Image_Style_Transfer_CVPR_2016_paper.pdf) to generate an artistic image. Using a provided template pre-trained on ImageNet, we want to extract the *style* of a first image (left) with the network and apply it to the *content* of a second image (right):

![Content and style, Mona-Lisa](https://pax.ulaval.ca/static/GIF-4101-7005/images/content_style_mona.png)

The objective is to create a hybrid image that contains both the style and the content of two images. This is a more visual exercise that will allow you to improve your intuition on how a convolution neural network works based on information from a scientific article. To give you an idea of the result, here is the hybrid image generated from the images presented above:

![Mona-Lisa transfer](https://pax.ulaval.ca/static/GIF-4101-7005/images/style_transfer_mona.gif)

The VGG19 convolution neural network is composed of 2 groups: the *features* and the classification layers. The style corresponds to the result of passing the filters of the *features* layers on the input image, while the content corresponds to the pixel values of the second image. 

## Q3A
Le transfert de style ne demande que deux images comme jeu de données pour son application:
- Une image qui contient un style que vous souhaitez extraire (`style_image`);
- Une image de contenu sur laquelle vous souhaitez appliquer le style (`content_image`).

L'objectif du réseau est d'optimiser les pixels de l'image hybride en pondérant le style et le contenu des images sources. Il est à noter que le réseau utilisé, VGG19, doit préalablement être entraîné sur un très grand nombre de données. Toutefois, puisqu'on utilise un réseau préentraîné, vous n'avez pas besoin de toutes ces images pour son entraînement.

La première étape du problème est de télécharger vos images de *style* et de *contenu*. Avec l'aide de la fonction `fetch_image`, vous pouvez télécharger vos propres images à partir de votre Google Drive. Pour se faire, téléversez une image de style ainsi qu'une image de contenu sur votre Google Drive et partagez-les publiquement en créant un lien URL de partage. Le lien aura la forme suivante:

`https://drive.google.com/file/d/<FILE_ID>/view?usp=sharing`

Copiez-collez le `<FILE_ID>` et passez-le comme chaîne de caractères en entrée de la fonction `fetch_image('<FILE_ID>')` pour télécharger votre image dans le notebook.

Quelques exemples d'images de contenu:
- [Great Sea Turtle](https://drive.google.com/file/d/11c650QrD0vP7le1EHiZ5nkjRuoUYmF6H/view?usp=sharing) (`<FILE_ID> : 11c650QrD0vP7le1EHiZ5nkjRuoUYmF6H`)
- [Tuebingen](https://drive.google.com/file/d/11ec7XKIPQXVq6jq0Swq96abJ3t4r6JQV/view?usp=sharing) (`<FILE_ID> : 11ec7XKIPQXVq6jq0Swq96abJ3t4r6JQV`)
- [Grace Hopper](https://drive.google.com/file/d/11hj6wRTK3LvfNH1H2eGZCRAFA_h-f3Ag/view?usp=sharing) (`<FILE_ID> : 11hj6wRTK3LvfNH1H2eGZCRAFA_h-f3Ag`)

Quelques exemples d'images de style:
- [The Great Wave off Kanagawa](https://drive.google.com/file/d/11lRkyOtVCSZFrYT5r44y1rYXlywOmdaU/view?usp=sharing) (`<FILE_ID> : 11lRkyOtVCSZFrYT5r44y1rYXlywOmdaU`)
- [Kadinsky](https://drive.google.com/file/d/11utiecLh-3JQspwfOVHowkoWOHsD4Zx5/view?usp=sharing) (`<FILE_ID> : 11utiecLh-3JQspwfOVHowkoWOHsD4Zx5`)
- [Van Gogh](https://drive.google.com/file/d/11vgRvxUxwh8Q5uwaD8O9aYKO7yucm57A/view?usp=sharing) (`<FILE_ID> : 11vgRvxUxwh8Q5uwaD8O9aYKO7yucm57A`)

## Q3A
Style transfer requires only two images as dataset for its application:
- An image that contains a style you want to extract (`style_image`);
- A content image on which you want to apply the style (`content_image`).

The objective of the network is to optimize the pixels of the hybrid image by weighting the style and content of the source images. It should be noted that the network used, VGG19, must first be trained on a very large amount of data. However, since a pre-trained network is used, you do not need all these images for its training.

The first step in the problem is to download your *style* and *content* images. With the help of the `fetch_image` function, you can upload your own from your Google Drive. To do so, upload a style image and a content image to your Google Drive and share them publicly by creating a share URL link. The link will look like this:

`https://drive.google.com/file/d/<FILE_ID>/view?usp=sharing`

Copy and paste the `<FILE_ID>` and pass it as a string as input to the `fetch_image('<FILE_ID>')` function to upload your image to the notebook.

Some examples of content images:
- [Great Sea Turtle](https://drive.google.com/file/d/11c650QrD0vP7le1EHiZ5nkjRuoUYmF6H/view?usp=sharing) (`<FILE_ID> : 11c650QrD0vP7le1EHiZ5nkjRuoUYmF6H`)
- [Tuebingen](https://drive.google.com/file/d/11ec7XKIPQXVq6jq0Swq96abJ3t4r6JQV/view?usp=sharing) (`<FILE_ID> : 11ec7XKIPQXVq6jq0Swq96abJ3t4r6JQV`)
- [Grace Hopper](https://drive.google.com/file/d/11hj6wRTK3LvfNH1H2eGZCRAFA_h-f3Ag/view?usp=sharing) (`<FILE_ID> : 11hj6wRTK3LvfNH1H2eGZCRAFA_h-f3Ag`)

Some examples of style images:
- [The Great Wave off Kanagawa](https://drive.google.com/file/d/11lRkyOtVCSZFrYT5r44y1rYXlywOmdaU/view?usp=sharing) (`<FILE_ID> : 11lRkyOtVCSZFrYT5r44y1rYXlywOmdaU`)
- [Kadinsky](https://drive.google.com/file/d/11utiecLh-3JQspwfOVHowkoWOHsD4Zx5/view?usp=sharing) (`<FILE_ID> : 11utiecLh-3JQspwfOVHowkoWOHsD4Zx5`)
- [Van Gogh](https://drive.google.com/file/d/11vgRvxUxwh8Q5uwaD8O9aYKO7yucm57A/view?usp=sharing) (`<FILE_ID> : 11vgRvxUxwh8Q5uwaD8O9aYKO7yucm57A`)

### Patron de code réponse à l'exercice Q3A

### Q3A answer code template

In [ ]:
# Avec la fonction fetch_image, téléchargez vos propres images.
# Pour se faire, ajoutez vos images sur votre Google Drive en les
# téléversant et partagez-les avec un lien URL. Dans ce lien, 
# vous retrouverez le <FILE_ID> qu'il faut copier-coller dans la
# fonction fetch_image.
# Exemple: https://drive.google.com/file/d/<FILE_ID>/view?usp=sharing
# With the fetch_image function, upload your own images.
# To do so, add your images to your Google Drive by
# uploading them and share them with a URL link. In this link, 
# you will find the <FILE_ID> that you need to copy and paste into the
# function fetch_image.
# Example: https://drive.google.com/file/d/<FILE_ID>/view?usp=sharing

# *** TODO ***
# Télécharger une image contenant le style à extraire
# Download an image containing the style to extract
style_image_file_id = ""
# ******
style_image = fetch_image(style_image_file_id)

# *** TODO **
# Télécharger une image sur laquelle appliquer le style
# Download an image on which to apply the style
content_image_file_id = ""
# ******
content_image = fetch_image(content_image_file_id)

images = {STYLE_IMAGE:style_image,
          CONTENT_IMAGE:content_image}

# Afficher les 2 images côte-à-côte
# Display the 2 images side by side
plt.figure(figsize=(15,15))

# Affichage du style_image
# Displaying the image_style
plt.subplot(1, 2, 1)
plt.imshow(images[STYLE_IMAGE])

# Affichage du content_image
# Displaying the content_image
plt.subplot(1, 2, 2)
plt.imshow(images[CONTENT_IMAGE])

### Entrez votre solution à Q3A dans la cellule ci-dessous

### Enter your answer to Q3A in the cell below.

<div class="feedback-cell" style="background: rgba(100 , 100 , 100 , 0.4)">
                <h3>Votre soumission a été enregistrée!</h3>
                <ul>
                    <li>notez qu'il n'y a <strong>pas</strong> de correction automatique pour cet exercice&puncsp;;</li>
                    <li>par conséquent, votre note est <strong>actuellement</strong> zéro&puncsp;;</li>
                    <li>elle sera cependant ajustée par le professeur dès que la correction manuelle sera complétée&puncsp;;</li>
                    <li>vous pouvez soumettre autant de fois que nécessaire jusqu'à la date d'échéance&puncsp;;</li>
                    <li>mais évitez de soumettre inutilement.</li>
                </ul>
            </div><p class="alert alert-warning"><strong>ATTENTION</strong>: cette soumission a été effectuée <strong>après</strong> l'échéance, elle ne sera <strong>pas</strong> considérée.</p>

In [ ]:
# Avec la fonction fetch_image, téléchargez vos propres images.
# Pour se faire, ajoutez vos images sur votre Google Drive en les
# téléversant et partagez-les avec un lien URL. Dans ce lien, 
# vous retrouverez le <FILE_ID> qu'il faut copier-coller dans la
# fonction fetch_image.
# Exemple: https://drive.google.com/file/d/<FILE_ID>/view?usp=sharing
# With the fetch_image function, upload your own images.
# To do so, add your images to your Google Drive by
# uploading them and share them with a URL link. In this link, 
# you will find the <FILE_ID> that you need to copy and paste into the
# function fetch_image.
# Example: https://drive.google.com/file/d/<FILE_ID>/view?usp=sharing

# *** TODO ***
# Télécharger une image contenant le style à extraire
# Download an image containing the style to extract
style_image_file_id = "1hmNO6kDm1Bc3rgA2x1PGYi8BB_i4ABXp"
# ******
style_image = fetch_image(style_image_file_id)

# *** TODO **
# Télécharger une image sur laquelle appliquer le style
# Download an image on which to apply the style
content_image_file_id = "1GYQx4xI8pPjH9AUaveDFGNWkPyGC1IMH"
# ******
content_image = fetch_image(content_image_file_id)

images = {STYLE_IMAGE:style_image,
          CONTENT_IMAGE:content_image}

# Afficher les 2 images côte-à-côte
# Display the 2 images side by side
plt.figure(figsize=(15,15))

# Affichage du style_image
# Displaying the image_style
plt.subplot(1, 2, 1)
plt.imshow(images[STYLE_IMAGE])

# Affichage du content_image
# Displaying the content_image
plt.subplot(1, 2, 2)
plt.imshow(images[CONTENT_IMAGE])

## Q3B
#### Prétraitement
Le prétraitement des images est nécessaire pour s'assurer que celles-ci aient les mêmes caractéristiques (taille, intensité moyenne, etc.) que celles des images utilisées pour l'entraînement. Il est également utilisé pour faire de l'augmentation de jeu de données, en ajoutant de la diversité dans le jeu d'entraînement pour augmenter le nombre d'images disponibles. Dans le cas du transfert de style, on l'utilise pour que les images respectent la même distribution que pour l'entraînement du réseau VGG19 utilisé.

Puisqu'un réseau préentraîné est utilisé pour le transfert de style, il est important d'appliquer les mêmes paramètres de normalisation que ceux utilisés pour l'entraînement. Le CNN ayant été entraîné sur *ImageNet*, on applique les mêmes paramètres que dans la [documentation](https://pytorch.org/vision/stable/models.html). Il est à noter que ces paramètres représentent la moyenne et la déviation standard pour chaque canal de l'image. Une image standard de type RGB dispose de 3 canaux (Red, Green, Blue).
* ImageNet_mean = [0.485, 0.456, 0.406]
* ImageNet_std = [0.229, 0.224, 0.225]

#### Post-traitement
Le post-traitement est nécessaire pour s'assurer que les images qui sont générées par le réseau de neurones respectent les propriétés naturelles d'une image réelle pour pouvoir être affichées avec *Matplotlib*. Une image standard de type RGB contient trois canaux de couleurs qui sont composés de pixels. L'intensité de la couleur de chacun des pixels se trouve dans une plage [0,1]. Toutefois, rien ne garantit que les pixels inférés par le réseau de neurones respecteront cette plage. En effet, comme le réseau VGG19 fait usage de la fonction d'activation sigmoïde, les pixels en sortie sont contenus entre [-1,1] et doivent donc être ramenés entre [0,1]. Il va de même pour la normalisation. Comme le réseau VGG19 a été préentraîné sur ImageNet avec des paramètres de normalisation spécifique à ce jeu de données, on normalise les images de style et de contenu de la même manière en prétraitement. Toutefois, afin d'afficher l'image hybride, il est important de renverser la normalisation pour obtenir un résultat visuellement intéressant.

#### Objectif
À partir du patron de prétraitement qui vous est donné et de la classe maison `AddDimension` qui vous permet d'ajouter une dimension, vous devez implémenter les transformations inverses (post-traitement) afin d'annuler les transformations qui ont été faites en amont de l'optimisation. Pour se faire, vous devez implémenter les modules de transformations suivants:
1. `RemoveDimension`: Retirer la dimension *B* de la *batch_size* (taille de lot). On veut que le format passe de (B,C,H,W) -> (C,H,W). Utilisez la fonction *PyTorch* `squeeze`.
2. `DeNormalize`: Retirer la normalisation en appliquant son inverse sur les valeurs des pixels de l'image. On veut annuler l'effet de *ImageNet_mean* et de *ImageNet_std* Vous devez faire les manipulations manuellement.
3. `Clamp`: Fixer les valeurs des pixels de l'image dans les bornes [0,1]. Utilisez la fonction *PyTorch* `clamp`.
4. `Permute`: Faire une permutation de l'ordre des dimensions pour permettre à la librairie Matplotlib de lire l'image. On veut que le format passe de (C,H,W) -> (H,W,C). Utilisez la fonction *PyTorch* `permute`.

## Q3B
#### Preprocessing
Image preprocessing is necessary to ensure that the images have the same characteristics as the images used for training. With this preprocessing, we aim to bring the images to the same dimension, normalize the intensities, etc. It is also used to increase the data set, by adding diversity to the training set and thus multiply the number of images available. In the case of style transfer, it is used so that the images respect the same distribution as for the training of the VGG19 network you will use.

Since a pre-trained network is used for the style transfer, it is important to apply the same normalization parameters as those used for training. Since the CNN was trained on *ImageNet*, we apply the same parameters as in the [documentation](https://pytorch.org/vision/stable/models.html). Note that these parameters represent the mean and standard deviation for each channel of the image. A standard RGB image has 3 channels (Red, Green, Blue).
* ImageNet_mean = [0.485, 0.456, 0.406]
* ImageNet_std = [0.229, 0.224, 0.225]

#### Postprocessing
Postprocessing is necessary to ensure that the images that are generated by the neural network respect the natural properties of a real image to be displayed with *Matplotlib*. A standard RGB image contains three color channels that are composed of pixels. The color intensity of each pixel is in a range [0,1]. However, there is no guarantee that the pixels inferred by the neural network will respect this range. Indeed, since the VGG19 network makes use of the sigmoid activation function, the output pixels are in the range [-1,1] and need to be brought back to [0,1]. The same applies to the normalization. Since the VGG19 network has been pre-trained on ImageNet with normalization parameters specific to this dataset, we normalize the style and content images in the same way in preprocessing. However, in order to display the hybrid image, it is important to reverse the normalization if we want to have a visually interesting result.

#### Objective
From the given preprocessing pattern and the custom class `AddDimension` which allows you to add a dimension, you have to implement the reverse transformations (postprocessing) in order to undo the transformations that have been done before the optimization. To do this, you need to implement the following transformation modules:
1. `RemoveDimension`: Remove the *B* dimension from the *batch_size*. We want the format to change from (B,C,H,W) -> (C,H,W). Use the *PyTorch* function `squeeze`.
2. `DeNormalize`: Remove the normalization by applying its inverse to the pixel values of the image. We want to cancel the effect of *ImageNet_mean* and *ImageNet_std*.
3. `Clamp`: Set the values of the pixels of the image in the bounds [0,1]. Use the *PyTorch* `clamp` function.
4 `Permute`: Permute the order of the dimensions to allow the Matplotlib library to read the image. We want the format to change from (C,H,W) -> (H,W,C). Use the *PyTorch* function `permute`. 

### Patron de code réponse à l'exercice Q3B

### Q3B answer code template

In [ ]:
# Classe de transformation Custom pour ajouter un channel à la position "dim".
# Cette classe vous est donnée comme exemple pour l'implémentation des autres transformations.
# Custom transformation class to add a channel at the "dim" position.
# This class is given as an example for the implementation of other transformations.
class AddDimension(object):
    def __init__(self, dim):
        self.dim = dim

    def __call__(self, x):
        """
        Args:
            tensor (Tensor): Tensor image of size (C,H,W).

        Returns:
            tensor (Tensor): Tensor image with an added channel, now of size (1,C,H,W).
        """

        new_x = x.unsqueeze(self.dim)
        return new_x
    
# Étapes de prétraitement
# 1. Redimensionner l'images à la taille désirée -> (3, 256, 256)
# 2. Transformer l'image PIL en tenseur
# 3. Appliquer la normalisation ImageNet
# 4. Ajouter une dimension pour PyTorch (C,H,W) -> (B,C,H,W)
#    où B est la taille de batch (lot).
#
# Preprocessing steps
# Resize the image to the desired size -> (3, 256, 256)
# 2. Transform the PIL image into a tensor
# 3. Apply ImageNet normalization
# 4. Add a dimension for PyTorch (C,H,W) -> (B,C,H,W)
# where B is the batch size.
preprocessing = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)),
                                    transforms.ToTensor(), 
                                    transforms.Normalize(mean=IMAGENET_MEAN,
                                                         std=IMAGENET_STD),
                                    AddDimension(0),
                                   ])


# Puisque PyTorch travaille sur des lots (batch)
# d'images, une dimension supplémentaire (B) est
# ajoutée pour son fonctionnement. L'image en entrée
# passe donc de la taille (3, 256, 256) à la taille
# (B, 3, 256, 256) où B, ici, est de taille 1, car il
# n'y a qu'une seule image par lot.
# 
# Since PyTorch works on batches of images,
# an extra dimension (B) is added for its operation. The input
# image goes from the size (3, 256, 256) to the size
# (B, 3, 256, 256) where B, here, is of size 1, because
# there is only one image per batch.
#
# Toutefois, afin d'afficher l'image hybride, il est important
# de retirer cette dimension supplémentaire, car les outils
# d'affichage s'attendent à afficher une image unique
# Cette prochaine classe doit donc vous permettre de
# retirer cette dimension supplémentaire. 
#
# However, in order to display the hybrid image, it is important
# to remove this extra dimension, because the display tools
# expect to display a single image. 
# This next class should allow you to 
# remove this extra dimension. 
class RemoveDimension(object):
    def __init__(self, dim):
        self.dim = dim

    def __call__(self, x):
        """
        Args:
            x (Tensor): Tensor image of size (1,C,H,W).

        Returns:
            new_x (Tensor): Tensor image with the removed channel, now of size (C,H,W).
        """

        # *** TODO ***
        # Implémentation d'une transformation Custom
        # pour retirer un channel à la position "dim"
        # Utilisez la fonction Pytorch Squeeze()
        # Implementation of a Custom transformation
        # to remove a channel at the "dim" position
        # Use the Pytorch Squeeze() function
        return new_x  # Retourne le x transformé / return the transformed x
        # ******

# Comme le réseau VGG19 a été pré-entraîné sur ImageNet
# avec des paramètres de normalisation spécifique à ce
# jeu de données, on assume qu'il sera plus performant sur
# une nouvelle distribution d'images si celle-ci partage
# également cette normalisation. Ainsi, pour l'extraction
# des features de style et de contenu, le modèle VGG19 doit
# travailler sur des images normalisées.
#
# As the VGG19 network has been pre-trained on ImageNet
# with specific normalization parameters for this dataset, 
# it is assumed that it will perform better on a new
# image distribution if it shares this normalization. Thus, for the extraction
# of style and content features, the VGG19 model must
# work on normalized images.
#
# Toutefois, afin d'afficher l'image hybride, il est
# important de retirer la normalisation si on 
# souhaite avoir un résultat visuellement intéressant,
# car la normalisation a un impact sur la distribution
# des valeurs de pixels dans l'image. La classe suivante
# doit vous permettre d'appliquer l'inverse de la
# normalisation.
#
# However, in order to display the hybrid image, it is
# important to remove the normalization 
# to get a visually interesting result,
# because normalization has an impact on the distribution
# of pixel values in the image. The following class
# should allow you to apply the inverse of the
# normalization.
class DeNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, x):
        """
        Args:
            x (Tensor): Tensor image of shape (C,H,W).
        Returns:
            new_x (Tensor): DeNormalized tensor image (C,H,W).
        """
        
        # *** TODO ***
        # Implémentation d'une transformation Custom
        # pour appliquer l'inverse de la normalisation.
        # Vous devez implémenter cette fonction manuellement
        # en utilisant des opérations sur les tenseurs.
        #
        # Implementation of a custom transformation 
        # to apply the inverse normalization.
        # You must create this function manually using
        # tensor operations.
        return new_x # Retourne le x transformé / return the transformed x
        # ******


# Pour que les images s'affichent, la valeur des pixels doit
# être retournée entre [0,1]. La classe suivante doit vous permettre de borner
# les valeurs des pixels de l'image hybride entre [0,1].
#
# For the images to be displayed, the pixel value must be between [0,1].
# The following class should allow you to bound
# the pixel values of the hybrid image between [0,1].
class Clamp(object):
    def __init__(self, min, max):
        self.min = float(min)
        self.max = float(max)

    def __call__(self, x):
        """
        Args:
            x (Tensor): Tensor of the image.

        Returns:
            new_x (Tensor): Tensor with values clipped within [0,1].
        """

        # *** TODO ***
        # Implémentation d'une transformation maison
        # pour borner les valeurs dans la plage [0, 1]
        # Utilisez la fonction PyTorch: Clamp()
        #
        # Implementation of a custom transformation
        # to clamp the values in the range [0, 1].
        # Use the PyTorch: Clamp() function
        return new_x # Retourne le x transformé / return the transformed x
        # ******

# Pour que la librairie Matplotlib puisse afficher le
# contenu des images, les canaux doivent être
# donnés dans le bon ordre. PyTorch utilise les images sous
# la forme (B,C,H,W) et Matplotlib doit recevoir les images sous 
# la forme (H,W,C). Comme le Permute est appelé après le
# RemoveDimension(), vous aurez ici, en entrée, un tenseur
# (C,H,W) que vous devez transformer dans la forme
# désirée pour l'affichage de Matplotlib.
#
# In order for the Matplotlib library to display the
# content of the images, the channels must
# be given in the right order. PyTorch uses images in the form
# (B,C,H,W) and Matplotlib must receive the images under 
# the form (H,W,C). As the Permute is called after the
# RemoveDimension(), you will have here, as input, a tensor
# (C,H,W) that you must transform into the
# desired form for the display of Matplotlib.
class Permute(object):
    def __init__(self, dims):
        self.dims = dims

    def __call__(self, x):
        """
        Args:
            x (Tensor): Tensor of the image.

        Returns:
            new_x (Tensor): Tensor of the image with permuted dimensions.
        """
        
        # *** TODO ***
        # Implémentation d'une transformation Custom pour
        # faire une permutation.
        # Utilisez la fonction PyTorch: Permute()
        #
        # Implementation of a custom permute tranformation.
        # Use the function Pytorch: Permute()
        return new_x
        # ******


# *** TODO ***
# transforms.Compose applique séquentiellement les
# transformations. Étapes de post-traitement (voir les modules précédents)
# 1. Retirer la 1ère dimension (B,C,H,W)->(C,H,W)
# 2. Appliquer l'inverse de la normalisation ImageNet
# 3. Permuter les dimension pour Matplotlib (C,H,W)->(H,W,C)
# 4. Clamp les valeurs des tenseurs entre [0,1]
# Vous devez ici passer les paramètres désirés dans l'appel des classes 
# de transformation.
#
# transforms.Compose applies sequentially the
# transforms. Postprocessing steps (see previous modules)
# 1. Remove the 1st dimension (B,C,H,W)->(C,H,W)
# 2. Apply the inverse of the ImageNet normalization
# 3. Swap dimensions for Matplotlib (C,H,W)->(H,W,C)
# 4. Clamp the tensor values between [0,1]
# Here you have to pass the desired parameters in the call of 
# transformation classes.
postprocessing = transforms.Compose([RemoveDimension(), 
                                     DeNormalize(mean=IMAGENET_MEAN,
                                                 std=IMAGENET_STD),
                                     Permute(),
                                     Clamp(),
                                    ])
# ******

# Le code suivant est donné pour permettre d'afficher
# certaines informations utiles qui vous permettront
# de comprendre si vos transformations post-traitement
# sont fonctionnelles.
#
# The following code is given to allow you to display
# some useful information that will allow you to
# understand if your postprocessing transformations
# are functional.

# Afficher les statistiques des images naturelles
# Display the statistics of natural images
for name, img in images.items():
    results['Name'].append(f'raw_{name}')
    results['Shape'].append(img.size)
    mean = numpy.mean(img)/255
    results['Mean'].append(mean)
    std = numpy.std(img)/255
    results['Std'].append(std)

# Appliquer le prétraitement sur les images
# Apply preprocessing on the images
pre_images = {}
for k,v in images.items():
    pre_images[k] = preprocessing(v)
    pre_images[k] = pre_images[k].to(DEVICE)

# Afficher les statistiques des images transformées
# Display the statistics of the transformed images
for name, img in pre_images.items():
    results['Name'].append(f'pre_{name}')
    results['Shape'].append(img.shape)
    results['Mean'].append(img.mean().item())
    results['Std'].append(img.std().item())

post_images = {}

# Appliquer le post-traitement sur les images
# Apply postprocessing to images
for name,img in pre_images.items():
    image = img.cpu().detach()
    post_images[name] = postprocessing(image)

# Afficher les statistiques des images transformées
# Display the statistics of the transformed images
for name, img in post_images.items():
    results['Name'].append(f'post_{name}')
    results['Shape'].append(img.shape)
    results['Mean'].append(img.mean().item())
    results['Std'].append(img.std().item())

# Affichage des résultats
# N.B. Bien que la taille de l'image ait été changée par le resize,
#      les valeurs de moyenne et de déviation standard devraient être
#      très proches avant le prétraitement et après le post-traitement.
# Displaying the results
# N.B. Although the size of the image has been changed by the resize,
# the mean and standard deviation values should be very close
# before preprocessing and after postprocessing.
df = pandas.DataFrame(results)
display.display(df)

### Entrez votre solution à Q3B dans la cellule ci-dessous

### Enter your answer to Q3B in the cell below.

In [ ]:
# Classe de transformation Custom pour ajouter un channel à la position "dim".
# Cette classe vous est donnée comme exemple pour l'implémentation des autres transformations.
# Custom transformation class to add a channel at the "dim" position.
# This class is given as an example for the implementation of other transformations.
class AddDimension(object):
    def __init__(self, dim):
        self.dim = dim

    def __call__(self, x):
        """
        Args:
            tensor (Tensor): Tensor image of size (C,H,W).

        Returns:
            tensor (Tensor): Tensor image with an added channel, now of size (1,C,H,W).
        """

        new_x = x.unsqueeze(self.dim)
        return new_x
    
# Étapes de prétraitement
# 1. Redimensionner l'images à la taille désirée -> (3, 256, 256)
# 2. Transformer l'image PIL en tenseur
# 3. Appliquer la normalisation ImageNet
# 4. Ajouter une dimension pour PyTorch (C,H,W) -> (B,C,H,W)
#    où B est la taille de batch (lot).
#
# Preprocessing steps
# Resize the image to the desired size -> (3, 256, 256)
# 2. Transform the PIL image into a tensor
# 3. Apply ImageNet normalization
# 4. Add a dimension for PyTorch (C,H,W) -> (B,C,H,W)
# where B is the batch size.
preprocessing = transforms.Compose([transforms.Resize((IMG_SIZE, IMG_SIZE)),
                                    transforms.ToTensor(), 
                                    transforms.Normalize(mean=IMAGENET_MEAN,
                                                         std=IMAGENET_STD),
                                    AddDimension(0),
                                   ])

# Puisque PyTorch travaille sur des lots (batch)
# d'images, une dimension supplémentaire (B) est
# ajoutée pour son fonctionnement. L'image en entrée
# passe donc de la taille (3, 256, 256) à la taille
# (B, 3, 256, 256) où B, ici, est de taille 1, car il
# n'y a qu'une seule image par lot.
# 
# Since PyTorch works on batches of images,
# an extra dimension (B) is added for its operation. The input
# image goes from the size (3, 256, 256) to the size
# (B, 3, 256, 256) where B, here, is of size 1, because
# there is only one image per batch.
#
# Toutefois, afin d'afficher l'image hybride, il est important
# de retirer cette dimension supplémentaire, car les outils
# d'affichage s'attendent à afficher une image unique
# Cette prochaine classe doit donc vous permettre de
# retirer cette dimension supplémentaire. 
#
# However, in order to display the hybrid image, it is important
# to remove this extra dimension, because the display tools
# expect to display a single image. 
# This next class should allow you to 
# remove this extra dimension. 
class RemoveDimension(object):
    def __init__(self, dim):
        self.dim = dim

    def __call__(self, x):
        """
        Args:
            x (Tensor): Tensor image of size (1,C,H,W).

        Returns:
            new_x (Tensor): Tensor image with the removed channel, now of size (C,H,W).
        """

        # *** TODO ***
        # Implémentation d'une transformation Custom
        # pour retirer un channel à la position "dim"
        # Utilisez la fonction Pytorch Squeeze()
        # Implementation of a Custom transformation
        # to remove a channel at the "dim" position
        # Use the Pytorch Squeeze() function
        new_x = x.squeeze(self.dim)
        return new_x  # Retourne le x transformé / return the transformed x
        # ******

# Comme le réseau VGG19 a été pré-entraîné sur ImageNet
# avec des paramètres de normalisation spécifique à ce
# jeu de données, on assume qu'il sera plus performant sur
# une nouvelle distribution d'images si celle-ci partage
# également cette normalisation. Ainsi, pour l'extraction
# des features de style et de contenu, le modèle VGG19 doit
# travailler sur des images normalisées.
#
# As the VGG19 network has been pre-trained on ImageNet
# with specific normalization parameters for this dataset, 
# it is assumed that it will perform better on a new
# image distribution if it shares this normalization. Thus, for the extraction
# of style and content features, the VGG19 model must
# work on normalized images.
#
# Toutefois, afin d'afficher l'image hybride, il est
# important de retirer la normalisation si on 
# souhaite avoir un résultat visuellement intéressant,
# car la normalisation a un impact sur la distribution
# des valeurs de pixels dans l'image. La classe suivante
# doit vous permettre d'appliquer l'inverse de la
# normalisation.
#
# However, in order to display the hybrid image, it is
# important to remove the normalization 
# to get a visually interesting result,
# because normalization has an impact on the distribution
# of pixel values in the image. The following class
# should allow you to apply the inverse of the
# normalization.
class DeNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, x):
        """
        Args:
            x (Tensor): Tensor image of shape (C,H,W).
        Returns:
            new_x (Tensor): DeNormalized tensor image (C,H,W).
        """
        
        # *** TODO ***
        # Implémentation d'une transformation Custom
        # pour appliquer l'inverse de la normalisation.
        # Vous devez implémenter cette fonction manuellement
        # en utilisant des opérations sur les tenseurs.
        #
        # Implementation of a custom transformation 
        # to apply the inverse normalization.
        # You must create this function manually using
        # tensor operations.
        mean = numpy.array(self.mean)
        std = numpy.array(self.std)
        u = transforms.Normalize(mean=-mean/std, std=1/std)
        new_x = u(x)
        return new_x # Retourne le x transformé / return the transformed x
        # ******


# Pour que les images s'affichent, la valeur des pixels doit
# être retournée entre [0,1]. La classe suivante doit vous permettre de borner
# les valeurs des pixels de l'image hybride entre [0,1].
#
# For the images to be displayed, the pixel value must be between [0,1].
# The following class should allow you to bound
# the pixel values of the hybrid image between [0,1].
class Clamp(object):
    def __init__(self, min, max):
        self.min = float(min)
        self.max = float(max)

    def __call__(self, x):
        """
        Args:
            x (Tensor): Tensor of the image.

        Returns:
            new_x (Tensor): Tensor with values clipped within [0,1].
        """

        # *** TODO ***
        # Implémentation d'une transformation maison
        # pour borner les valeurs dans la plage [0, 1]
        # Utilisez la fonction PyTorch: Clamp()
        #
        # Implementation of a custom transformation
        # to clamp the values in the range [0, 1].
        # Use the PyTorch: Clamp() function
        new_x = torch.clamp(x, min=self.min, max=self.max)
        return new_x # Retourne le x transformé / return the transformed x
        # ******

# Pour que la librairie Matplotlib puisse afficher le
# contenu des images, les canaux doivent être
# donnés dans le bon ordre. PyTorch utilise les images sous
# la forme (B,C,H,W) et Matplotlib doit recevoir les images sous 
# la forme (H,W,C). Comme le Permute est appelé après le
# RemoveDimension(), vous aurez ici, en entrée, un tenseur
# (C,H,W) que vous devez transformer dans la forme
# désirée pour l'affichage de Matplotlib.
#
# In order for the Matplotlib library to display the
# content of the images, the channels must
# be given in the right order. PyTorch uses images in the form
# (B,C,H,W) and Matplotlib must receive the images under 
# the form (H,W,C). As the Permute is called after the
# RemoveDimension(), you will have here, as input, a tensor
# (C,H,W) that you must transform into the
# desired form for the display of Matplotlib.
class Permute(object):
    def __init__(self, dims):
        self.dims = dims

    def __call__(self, x):
        """
        Args:
            x (Tensor): Tensor of the image.

        Returns:
            new_x (Tensor): Tensor of the image with permuted dimensions.
        """
        
        # *** TODO ***
        # Implémentation d'une transformation Custom pour
        # faire une permutation.
        # Utilisez la fonction PyTorch: Permute()
        #
        # Implementation of a custom permute tranformation.
        # Use the function Pytorch: Permute()
        new_x = x.permute(1, 2, 0)
        return new_x
        # ******


# *** TODO ***
# transforms.Compose applique séquentiellement les
# transformations. Étapes de post-traitement (voir les modules précédents)
# 1. Retirer la 1ère dimension (B,C,H,W)->(C,H,W)
# 2. Appliquer l'inverse de la normalisation ImageNet
# 3. Permuter les dimension pour Matplotlib (C,H,W)->(H,W,C)
# 4. Clamp les valeurs des tenseurs entre [0,1]
# Vous devez ici passer les paramètres désirés dans l'appel des classes 
# de transformation.
#
# transforms.Compose applies sequentially the
# transforms. Postprocessing steps (see previous modules)
# 1. Remove the 1st dimension (B,C,H,W)->(C,H,W)
# 2. Apply the inverse of the ImageNet normalization
# 3. Swap dimensions for Matplotlib (C,H,W)->(H,W,C)
# 4. Clamp the tensor values between [0,1]
# Here you have to pass the desired parameters in the call of 
# transformation classes.
postprocessing = transforms.Compose([RemoveDimension(0), 
                                     DeNormalize(mean=IMAGENET_MEAN,
                                                 std=IMAGENET_STD),
                                     Permute(0),
                                     Clamp(0, 1),
                                    ])
# ******

# Le code suivant est donné pour permettre d'afficher
# certaines informations utiles qui vous permettront
# de comprendre si vos transformations post-traitement
# sont fonctionnelles.
#
# The following code is given to allow you to display
# some useful information that will allow you to
# understand if your postprocessing transformations
# are functional.

# Afficher les statistiques des images naturelles
# Display the statistics of natural images
for name, img in images.items():
    results['Name'].append(f'raw_{name}')
    results['Shape'].append(img.size)
    mean = numpy.mean(img)/255
    results['Mean'].append(mean)
    std = numpy.std(img)/255
    results['Std'].append(std)

# Appliquer le prétraitement sur les images
# Apply preprocessing on the images
pre_images = {}
for k,v in images.items():
    pre_images[k] = preprocessing(v)
    pre_images[k] = pre_images[k].to(DEVICE)

# Afficher les statistiques des images transformées
# Display the statistics of the transformed images
for name, img in pre_images.items():
    results['Name'].append(f'pre_{name}')
    results['Shape'].append(img.shape)
    results['Mean'].append(img.mean().item())
    results['Std'].append(img.std().item())

post_images = {}

# Appliquer le post-traitement sur les images
# Apply postprocessing to images
for name,img in pre_images.items():
    image = img.cpu().detach()
    post_images[name] = postprocessing(image)

# Afficher les statistiques des images transformées
# Display the statistics of the transformed images
for name, img in post_images.items():
    results['Name'].append(f'post_{name}')
    results['Shape'].append(img.shape)
    results['Mean'].append(img.mean().item())
    results['Std'].append(img.std().item())

# Affichage des résultats
# N.B. Bien que la taille de l'image ait été changée par le resize,
#      les valeurs de moyenne et de déviation standard devraient être
#      très proches avant le prétraitement et après le post-traitement.
# Displaying the results
# N.B. Although the size of the image has been changed by the resize,
# the mean and standard deviation values should be very close
# before preprocessing and after postprocessing.
df = pandas.DataFrame(results)
display.display(df)

## Q3C
Maintenant que nous avons les données pour l'entraînement et qu'elles sont formatées et normalisées, il faut télécharger le modèle préentraîné VGG19 de la librairie PyTorch. Puisque nous n'avons pas besoin des couches de classification, seules les couches de *features* sont stockées dans la variable `vgg`. Pour ce faire, affichez les noms des modules et couches du modèle et ne sélectionnez que ce qui correspond aux couches de *features*. Vous ne voulez pas conserver les couches de classification, car elles ne sont pas utiles pour la suite du problème. Ensuite, vous devez geler les paramètres du réseau, car ils ne seront pas modifiés.

## Q3C
Now that we have the data for training and it is formatted and normalized, we need to download the VGG19 pre-trained model from the PyTorch library. Since we don't need the classification layers, we only store the *features* layers in the variable `vgg`. To do this, display the names of the modules and layers in the model and select only those that correspond to the *features* layers. You don't want to keep the classification layers, as they are not useful for the rest of the problem. Next, you need to freeze the network settings, as they will not be modified.

### Patron de code réponse à l'exercice Q3C

### Q3C answer code template

In [ ]:
# *** TODO ***
# Télécharger la portion "features" du VGG19
# Nous n'avons pas besoin des couches de classification.
# !!! Veuillez passer en paramètre progress=False dans la  !!!
# !!! fonction. Autrement, l'exécution vous retournera une !!!
# !!! erreur.                                              !!!
# Geler les couches pré-entraînées

# Download the features portion of the VGG19
# We don't need the classification layers.
# !!! Please pass in the progress=False parameter in the !!!
# !!! function. Otherwise, the execution will return an !!!
# !!! error. !!!
# Freeze pre-trained layers
# ******

# Si GPU disponible, monter le modèle sur le GPU
# If GPU available, mount the model on the GPU
vgg.to(DEVICE)

### Entrez votre solution à Q3C dans la cellule ci-dessous

### Enter your answer to Q3C in the cell below.

In [ ]:
# *** TODO ***
# Télécharger la portion "features" du VGG19
# Nous n'avons pas besoin des couches de classification.
# !!! Veuillez passer en paramètre progress=False dans la  !!!
# !!! fonction. Autrement, l'exécution vous retournera une !!!
# !!! erreur.                                              !!!
# Geler les couches pré-entraînées
#
# Download the features portion of the VGG19
# We don't need the classification layers.
# !!! Please pass in the progress=False parameter in the !!!
# !!! function. Otherwise, the execution will return an !!!
# !!! error. !!!
# Freeze pre-trained layers
# ******
vgg = models.vgg19(pretrained=True, progress=False).features
for name, param in vgg.named_parameters():
    param.requires_grad = False
    
# Si GPU disponible, monter le modèle sur le GPU
# If GPU available, mount the model on the GPU
vgg.to(DEVICE)

## Q3D
Afin de pouvoir développer les fonctions de perte (*loss*) et lancer la génération de l'image hybride, il faut tout d'abord initialiser les pondérations et calculer quelques paramètres importants. Vous devez donc:
1. Extraire les *features* de l'image de style avec la fonction `extract_features`;
2. Extraire les *features* de l'image de contenu avec la fonction `extract_features`;
3. Créer une copie de l'image de contenu (appelée image cible) pour permettre de l'ajuster itérativement tout en gardant une copie du contenu original.

Également, vous êtes invités à ajuster les pondérations et les paramètres $\alpha$ et $\beta$ pour rendre votre rendu le plus à votre goût possible. Notez bien que vous n'êtes pas évalué sur la qualité du rendu, mais on vous encourage fortement à tester d'autres configurations de paramètres pour bien en comprendre le fonctionnement.

## Q3D
In order to develop the loss functions and launch the generation of the hybrid image, you must first initialize the weights and calculate some important parameters. So you need to:
1. Extract the *features* from the style image with the `extract_features` function;
2. Extract the *features* from the content image with the `extract_features` function;
3. Create a copy of the content image (called the target image) to allow it to be adjusted iteratively while keeping a copy of the original content.

Also, you are invited to adjust the weights and the $\alpha$ and $\beta$ parameters to make your rendering as pleasing as possible. Note that you are not evaluated on the quality of the rendering, but you are strongly encouraged to test other parameter configurations to understand how they work.

### Patron de code réponse à l'exercice Q3D

### Q3D answer code template

In [ ]:
# ** TODO ***
# Extraire les features de l'image de style avec la fonction
# extract_features.
# Extract the features of the style image with the function
# extract_features.
# ******

# *** TODO ***
# Extraire les features de l'image de content avec la fonction
# extract_features.
# Extract the features of the content image with the function
# extract_features.
# ******

# Pré-calculer la matrice de Gram pour chaque couche de style
# Pre-compute the Gram matrix for each style layer
style_grams = {}
for layer in style_features:
    style_grams[layer] = gram_matrix(style_features[layer])

# *** TODO ***
# Création d'une image cible temporaire. Utilisez la fonction clone() de
# la librairie PyTorch. N'oubliez pas le gradient! Considérez également le
# device utilisé (CPU vs GPU). Il faut travailler sur une copie de
# l'image cible pour changer son style itérativement.
#
# Create a temporary target image. Use the clone() function of
# the PyTorch library. Don't forget the gradient! Also consider the
# device used (CPU vs GPU). You have to work on a copy of the
# the target image to change its style iteratively.
# ******

# Poids appliqués pour chaque couche de style
# Weights applied for each style layer 
# Valeurs par défaut / default values:
# 'conv1_1': 1.
# 'conv2_1': 0.75
# 'conv3_1': 0.2
# 'conv4_1': 0.2
# 'conv5_1': 0.2
style_layers_weights = {'conv1_1': 1.,
                        'conv2_1': 0.75,
                        'conv3_1': 0.2,
                        'conv4_1': 0.2,
                        'conv5_1': 0.2}

# Par défaut: content_weight = 1
# By default: content_weight = 1
content_weight = 1 

# Par défaut: style_weight = 1e7
# By default: style_weight = 1e7
style_weight = 1e7

### Entrez votre solution à Q3D dans la cellule ci-dessous

### Enter your answer to Q3D in the cell below.

In [ ]:
# ** TODO ***
# Extraire les features de l'image de style avec la fonction
# extract_features.
# Extract the features of the style image with the function
# extract_features.
# ******
style_features = extract_features(pre_images[STYLE_IMAGE], vgg)
# *** TODO ***
# Extraire les features de l'image de content avec la fonction
# extract_features.
# Extract the features of the content image with the function
# extract_features.
# ******
content_features = extract_features(pre_images[CONTENT_IMAGE], vgg)

# Pré-calculer la matrice de Gram pour chaque couche de style
# Pre-compute the Gram matrix for each style layer
style_grams = {}
for layer in style_features:
    style_grams[layer] = gram_matrix(style_features[layer])

# *** TODO ***
# Création d'une image cible temporaire. Utilisez la fonction clone() de
# la librairie PyTorch. N'oubliez pas le gradient! Considérez également le
# device utilisé (CPU vs GPU). Il faut travailler sur une copie de
# l'image cible pour changer son style itérativement.
#
# Create a temporary target image. Use the clone() function of
# the PyTorch library. Don't forget the gradient! Also consider the
# device used (CPU vs GPU). You have to work on a copy of the
# the target image to change its style iteratively.
# ******
target = pre_images[CONTENT_IMAGE].clone()
target.requires_grad = True

# Poids appliqués pour chaque couche de style
# Weights applied for each style layer 
# Valeurs par défaut / default values:
# 'conv1_1': 1.
# 'conv2_1': 0.75
# 'conv3_1': 0.2
# 'conv4_1': 0.2
# 'conv5_1': 0.2
style_layers_weights = {'conv1_1': 1.,
                        'conv2_1': 0.75,
                        'conv3_1': 0.2,
                        'conv4_1': 0.2,
                        'conv5_1': 0.2}

# Par défaut: content_weight = 1
# By default: content_weight = 1
content_weight = 1 

# Par défaut: style_weight = 1e7
# By default: style_weight = 1e7
style_weight = 1e7

## Q3E
Le transfert de style, comme vous pouvez le comprendre dans l'article de [Gatys et coll.](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Gatys_Image_Style_Transfer_CVPR_2016_paper.pdf), ne requière pas d'entraînement à proprement parler. En effet, puisqu'on utilise un réseau préentraîné et qu'on en gèle les poids, le réseau n'apprend rien. Toutefois, les pixels de l'image cible (hybride) doivent être optimisés pour contenir à la fois le *style* ainsi que le *contenu* désiré. On fait donc une descente de gradient avec fonction de perte pour minimiser la différence de contenu entre l'image originale et l'image générée, tout en intégrant le nouveau style à l'image générée.

Pour se faire, deux fonctions de pertes doivent être développées: une qui permet de mesurer la différence de contenu entre l'image cible et l'image de contenu (*perte de contenu*) ainsi qu'une seconde qui permet de mesurer la différence de style entre la matrice Gram de l'image cible et celle de l'image de style (*perte de style*). La somme pondérée de ces deux fonctions permet de générer la *perte totale* qui sera utilisée pour l'optimisation des pixels de l'image hybride.

1. Implémenter la fonction `calculate_content_loss` qui prend en paramètre le nom de la couche $l$ à évaluer et qui retourne la perte de contenu entre les paramètres de l'image cible ($F$) et les paramètres de l'image de contenu ($P$). La fonction de calcul de la perte est la suivante:
$$L_{content}(\vec{p},\vec{x},l)=\frac{1}{2}\sum_{i,j}(F_{i,j}^l-P_{i,j}^l)^2,$$ où la couche qui doit être utilisée pour calculer la perte de contenu est la 21e couche du modèle VGG19. Vous retrouverez le nom de cette couche dans le graphe du modèle.

2. Implémenter la fonction `calculate_style_loss` qui retourne la perte de style entre la matrice Gram de l'image cible ($G$) et la matrice Gram de l'image de style ($A$). La fonction de calcul de cette perte est: $$E_l=\frac{1}{4N_l^2M_l^2} \sum_{i,j}(G_{i,j}^l-A_{i,j}^l)^2,$$ où $M$ et $N$ sont des paramètres que vous devez extraire de votre compréhension de l'article, suivi de: $$L_{style}(\vec{a},\vec{x})=\sum_{l=0}^{L}w_{l}E_{l},$$ où $w_l$ est la pondération donnée à la couche $l$.

3. Implémenter la fonction `calculate_total_loss` qui retourne la perte totale pour l'itération actuelle. La fonction pour vous permettre de calculer cette perte est: $$L_{total}(\vec{a},\vec{p},\vec{x})=\alpha L_{content}(\vec{p},\vec{x})+\beta L_{style}(\vec{a},\vec{x})$$.

## Q3E
Style transfer, as you can read in the paper by [Gatys et al](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Gatys_Image_Style_Transfer_CVPR_2016_paper.pdf), does not require training per se. Indeed, since we use a pre-trained network and freeze the weights, the network will not learn anything. However, the pixels of the target (hybrid) image must be optimized to contain both the *style* and the desired *content*. We therefore perform a gradient descent with a loss function to minimize the difference in content between the original image and the generated image, while integrating the new style into the generated image.

To do this, two loss functions must be developed: one that measures the difference in content between the target image and the content image (*content loss*) and a second that measures the difference in style between the Gram matrix of the target image and the style image (*style loss*). The weighted sum of these two functions allows to generate the *total loss* which will be used for the optimization of the pixels of the hybrid image.

1. Implement the function `calculate_content_loss` which takes as parameter the name of the layer $l$ to be evaluated and returns the content loss between the parameters of the target image ($F$) and the parameters of the content image ($P$). The function for computing the loss is as follows:
$$L_{content}(\vec{p},\vec{x},l)=\frac{1}{2}\sum_{i,j}(F_{i,j}^l-P_{i,j}^l)^2$$, where the layer that is to be used to compute the content loss is the 21st layer of the VGG19 model. You will find the name of this layer in the model graph.

2. Implement the function `calculate_style_loss` which returns the style loss between the Gram matrix of the target image ($G$) and the Gram matrix of the style image ($A$). The function of calculation of this loss is: $$E_l=frac{1}{4N_l^2M_l^2} \sum_{i,j}(G_{i,j}^l-A_{i,j}^l)^2,$$ where $M$ and $N$ are parameters you need to extract from your understanding of the article, followed by: $$L_{style}(\vec{a},\vec{x})=\sum_{l=0}^{L}w_{l}E_{l},$$ where $w_l$ is the weight given to the $l$ layer.

3. Implement the function `calculate_total_loss` which returns the total loss for the current iteration. The function to allow you to calculate this loss is: $$L_{total}(\vec{a},\vec{p},\vec{x})=$alpha L_{content}(\vec{p},\vec{x})+$beta L_{style}(\vec{a},\vec{x})$$.

In [ ]:
# *** TODO ***
# Implémenter la fonction qui calcule la perte de contenu pour
# une couche donnée. La perte est calculée comme l'erreur quadratique
# entre les paramètres de l'image cible et les paramètres de l'image
# de contenu pour chaque couche.
#
# Implement the function that calculates the content loss for
# a given layer. The loss is calculated as the squared error
# between the parameters of the target image and the parameters
# of the content image for each layer.
def calculate_content_loss(layer_name):
    assert(layer_name in target_features.keys())
    assert(layer_name in content_features.keys())
    """
    Calculates the content loss between the target image features and
    the content image features.

    Args:
        layer_name (String) : Name of the layer to evaluate.

    Returns:
        tensor (Tensor): Tensor containing the loss of the squared mean
                         difference between the target and content layers.
    """
    return content_loss
# ******

# *** TODO ***
# Implémenter la fonction qui calcule la perte de style pour une couche donnée.
# Cette perte de style est calculée par l'erreur entre la matrice Gram
# de contenu et la matrice Gram de style, pondérée par le poids donné à chaque couche.
#
# Implement the function that calculates the style loss for a given layer.
# This style loss is calculated as the error between the content Gram
# matrix and the style Gram matrix, weighted by the weight given to each layer.
def calculate_style_loss(weight_layer, target_gram, style_gram, target_feature):
    """
    Calculates the style loss between the Gram matrix of the image features and
    the Gram matrix of the content image features.

    Args:
        weight_layer (Float) : weighting for the current layer (w_l).
        target_gram (Tensor) : Gram matrix of the target image (G).
        style_gram (Tensor) : Gram matrix of the style image (A).
        target_feature (Tensor) : tensor containing the target feature for
                                  the current layer.

    Returns:
        style_loss (Float): computed style loss for the current layer. 
    """
    return style_loss
# ******

# *** TODO ***
# Implémenter la fonction qui calcule la perte totale pour l'itération.
# La perte totale est calculée par la somme pondérée de la perte de contenu
# et la perte de style.
# Implement the function that calculates the total loss for the iteration.
# The total loss is calculated by the weighted sum of the content loss
# and the style loss.
def calculate_total_loss(content_weight, content_loss, style_weight, style_loss):
    """
    Calculates the total loss for the current iteration.

    Args:
        content_weight (Float) : Alpha weighting for the content.
        content_loss (Float) : Content loss.
        style_weight (Float) : Beta weighting for the style.
        style_loss (Float) : Total loss.

    Returns:
        total_loss (Float): computed total loss for the current iteration. 
    """
    return total_loss
# ******

# Nombre total d'itérations pour appliquer
# le transfert de style (Min: 2000 | Recommandé: 5000)
# Total number of iterations to apply
# style transfer (Min: 2000 | Recommended: 5000)
steps = 5000

# Fréquence de mise à jour de l'image (Valeur recommandée: 500)
# Image update frequency (Recommended value: 500)
show_image_every = 500

# Initialisation de l'optimiseur Adam. Puisqu'on modifie la cible,
# on l'applique directement sur les pixels de l'image (learning_rate = 3e-3).
# Initialization of the Adam optimizer. Since we modify the target,
# we apply it directly on the pixels of the image (learning_rate = 3e-3).
#
# [Gatys et coll, 2016] font usage de L-BFGS mais pour simplifier l'implémentation et accélérer
# la convergence vers des résultats visibles, Adam est plus approprié.
# [Gatys et al, 2016] make use of L-BFGS but to simplify implementation and accelerate
# convergence to visible results, Adam is more appropriate.
optimizer = optim.Adam([target], lr=3e-3)

for s in tqdm(range(1, steps+1)):

    # 1. Remise à zéro des gradients
    # 1. Reset the gradients to zero
    optimizer.zero_grad()
    
    # 2. Extraire les features de l'image cible
    # 2. Extract the features of the target image
    target_features = extract_features(target, vgg)
    
    # *** TODO ***
    # 3. Calculer la loss de contenu avec la fonction
    # calculate_content_loss. Vous devez retrouver le
    # nom de la couche dans le graphe du modèle.
    # 3. Calculate the content loss with the function
    # calculate_content_loss. You must find the
    # name of the layer in the model graph.
    layer_name = ""
    content_loss = calculate_content_loss(layer_name)
    # ******

    # 4. Calculer la loss de style en accumulant sa valeur pour chaque couche
    # 4. Calculate the style loss by accumulating its value for each layer
    style_loss = 0 # Initialiser la loss de style à zéro / Initialise style loss to zero
    for l_name, l_weight in style_layers_weights.items():
        
        # Extraire le contenu de la couche / Extract layer content
        target_feature = target_features[l_name]

        # Calculer la matrice de Gram du contenu / Compute content Gram matrix
        target_gram = gram_matrix(target_feature)

        # Extraire la matrice Gram pré-calculée pour le style
        # Extract the pre-computed Gram matrix for the style
        style_gram = style_grams[l_name]

        # Calculer la loss de style avec pondération pour 
        # la couche donnée avec la fonction calculate_style_loss().
        # Calculate the weighted syle loss for 
        # the given layer with the function calculate_style_loss().
        layer_style_loss = calculate_style_loss(l_weight,
                                                target_gram,
                                                style_gram,
                                                target_feature)

        # Accumuler la loss de style / Accumulate style loss
        style_loss += layer_style_loss
    
    # 5. Calculer la loss totale avec la fonction calculate_total_loss()
    # 5. Calculate the total loss with the function calculate_total_loss()
    total_loss = calculate_total_loss(content_weight,
                                      content_loss,
                                      style_weight,
                                      style_loss)
    
    # 6. Mettre à jour l'image cible
    # 6. Update the target image
    total_loss.backward()
    optimizer.step()
    
    # Afficher les images intermédiaires
    # Display intermediate images
    if  s % show_image_every == 0:
        # Appliquer le postprocessing sur les images
        # Apply postprocessing to the images
        plt.figure(figsize=(10,10))
        img = target.cpu().detach()
        img_post = postprocessing(img)
        plt.imshow(img_post)
        plt.axis('off')
        plt.show()

# Libère la cache sur le GPU *important sur un cluster de GPU*
# Free the cache on the GPU *important on a GPU cluster*.
torch.cuda.empty_cache()

### Entrez votre solution à Q3E dans la cellule ci-dessous

### Enter your answer to Q3E in the cell below.

In [ ]:
# *** TODO ***
# Implémenter la fonction qui calcule la perte de contenu pour
# une couche donnée. La perte est calculée comme l'erreur quadratique
# entre les paramètres de l'image cible et les paramètres de l'image
# de contenu pour chaque couche.
#
# Implement the function that calculates the content loss for
# a given layer. The loss is calculated as the squared error
# between the parameters of the target image and the parameters
# of the content image for each layer.
def calculate_content_loss(layer_name):
    assert(layer_name in target_features.keys())
    assert(layer_name in content_features.keys())
    """
    Calculates the content loss between the target image features and
    the content image features.

    Args:
        layer_name (String) : Name of the layer to evaluate.

    Returns:
        tensor (Tensor): Tensor containing the loss of the squared mean
                         difference between the target and content layers.
    """    
    content_loss = torch.sum((target_features[layer_name] - content_features[layer_name])**2)/2
    return content_loss
# ******

# *** TODO ***
# Implémenter la fonction qui calcule la perte de style pour une couche donnée.
# Cette perte de style est calculée par l'erreur entre la matrice Gram
# de contenu et la matrice Gram de style, pondérée par le poids donné à chaque couche.
#
# Implement the function that calculates the style loss for a given layer.
# This style loss is calculated as the error between the content Gram
# matrix and the style Gram matrix, weighted by the weight given to each layer.
def calculate_style_loss(weight_layer, target_gram, style_gram, target_feature):
    """
    Calculates the style loss between the Gram matrix of the image features and
    the Gram matrix of the content image features.

    Args:
        weight_layer (Float) : weighting for the current layer (w_l).
        target_gram (Tensor) : Gram matrix of the target image (G).
        style_gram (Tensor) : Gram matrix of the style image (A).
        target_feature (Tensor) : tensor containing the target feature for
                                  the current layer.

    Returns:
        style_loss (Float): computed style loss for the current layer. 
    """
    N_l = target_feature.size(dim=1)
    M_l = target_feature.size(dim=2) * target_feature.size(dim=3)
    E_l = torch.sum((target_gram - style_gram)**2)/(4*(N_l**2)*(M_l**2))
    style_loss = weight_layer*E_l
    return style_loss
# ******

# *** TODO ***
# Implémenter la fonction qui calcule la perte totale pour l'itération.
# La perte totale est calculée par la somme pondérée de la perte de contenu
# et la perte de style.
# Implement the function that calculates the total loss for the iteration.
# The total loss is calculated by the weighted sum of the content loss
# and the style loss.
def calculate_total_loss(content_weight, content_loss, style_weight, style_loss):
    """
    Calculates the total loss for the current iteration.

    Args:
        content_weight (Float) : Alpha weighting for the content.
        content_loss (Float) : Content loss.
        style_weight (Float) : Beta weighting for the style.
        style_loss (Float) : Total loss.

    Returns:
        total_loss (Float): computed total loss for the current iteration. 
    """
    total_loss = content_weight * content_loss + style_weight * style_loss
    return total_loss
# ******

# Nombre total d'itérations pour appliquer
# le transfert de style (Min: 2000 | Recommandé: 5000)
# Total number of iterations to apply
# style transfer (Min: 2000 | Recommended: 5000)
steps = 5000

# Fréquence de mise à jour de l'image (Valeur recommandée: 500)
# Image update frequency (Recommended value: 500)
show_image_every = 500

# Initialisation de l'optimiseur Adam. Puisqu'on modifie la cible,
# on l'applique directement sur les pixels de l'image (learning_rate = 3e-3).
# Initialization of the Adam optimizer. Since we modify the target,
# we apply it directly on the pixels of the image (learning_rate = 3e-3).
#
# [Gatys et coll, 2016] font usage de L-BFGS mais pour simplifier l'implémentation et accélérer
# la convergence vers des résultats visibles, Adam est plus approprié.
# [Gatys et al, 2016] make use of L-BFGS but to simplify implementation and accelerate
# convergence to visible results, Adam is more appropriate.
optimizer = optim.Adam([target], lr=3e-3)

for s in tqdm(range(1, steps+1)):

    # 1. Remise à zéro des gradients
    # 1. Reset the gradients to zero
    optimizer.zero_grad()
    
    # 2. Extraire les features de l'image cible
    # 2. Extract the features of the target image
    target_features = extract_features(target, vgg)
    
    # *** TODO ***
    # 3. Calculer la loss de contenu avec la fonction
    # calculate_content_loss. Vous devez retrouver le
    # nom de la couche dans le graphe du modèle.
    # 3. Calculate the content loss with the function
    # calculate_content_loss. You must find the
    # name of the layer in the model graph.
    layer_name = "conv4_2"
    content_loss = calculate_content_loss(layer_name)
    # ******

    # 4. Calculer la loss de style en accumulant sa valeur pour chaque couche
    # 4. Calculate the style loss by accumulating its value for each layer
    style_loss = 0 # Initialiser la loss de style à zéro / Initialise style loss to zero
    for l_name, l_weight in style_layers_weights.items():
        
        # Extraire le contenu de la couche / Extract layer content
        target_feature = target_features[l_name]

        # Calculer la matrice de Gram du contenu / Compute content Gram matrix
        target_gram = gram_matrix(target_feature)

        # Extraire la matrice Gram pré-calculée pour le style
        # Extract the pre-computed Gram matrix for the style
        style_gram = style_grams[l_name]

        # Calculer la loss de style avec pondération pour 
        # la couche donnée avec la fonction calculate_style_loss().
        # Calculate the weighted syle loss for 
        # the given layer with the function calculate_style_loss().
        layer_style_loss = calculate_style_loss(l_weight,
                                                target_gram,
                                                style_gram,
                                                target_feature)

        # Accumuler la loss de style / Accumulate style loss
        style_loss += layer_style_loss
    
    # 5. Calculer la loss totale avec la fonction calculate_total_loss()
    # 5. Calculate the total loss with the function calculate_total_loss()
    total_loss = calculate_total_loss(content_weight,
                                      content_loss,
                                      style_weight,
                                      style_loss)
    
    # 6. Mettre à jour l'image cible
    # 6. Update the target image
    total_loss.backward()
    optimizer.step()
    
    # Afficher les images intermédiaires
    # Display intermediate images
    if  s % show_image_every == 0:
        # Appliquer le postprocessing sur les images
        # Apply postprocessing to the images
        plt.figure(figsize=(10,10))
        img = target.cpu().detach()
        img_post = postprocessing(img)
        plt.imshow(img_post)
        plt.axis('off')
        plt.show()

# Libère la cache sur le GPU *important sur un cluster de GPU*
# Free the cache on the GPU *important on a GPU cluster*.
torch.cuda.empty_cache()

## Q3F
Pour ce cas du transfert de style, plusieurs paramètres peuvent être modifiés pour permettre de modifier le rendu de l'image hybride. La pondération pour chacune des couches de style peut être modifiée ainsi que les paramètres $\alpha$ et $\beta$. En fonction de votre compréhension de l'article et de vos tests, observez l'effet de chacun de ces paramètres.

Dans la cellule de réponse prévue à cet effet, veuillez répondre aux questions suivantes:
1. Quel est l'effet de pondérer différemment les couches sur le style?
2. Que se passe-t-il si on pondère plus fortement les premières couches?
3. Que se passe-t-il si on pondère plus fortement les dernières couches?
4. Quel est l'effet du paramètre $\alpha$?
5. Quel est l'effet du paramètre $\beta$?
6. Qu'est-ce qui est, selon vous, une bonne configuration des paramètres de pondération des poids des couches de style, $\alpha$ et $\beta$, et pourquoi?

## Q3F
For this case of style transfer, several parameters can be modified to allow to modify the rendering of the hybrid image. The weighting for each of the style layers can be modified as well as the $\alpha$ and $\beta$ parameters. Depending on your understanding of the article and your tests, observe the effect of each of these parameters.

In the answer cell provided, please answer the following questions:
1. What is the effect of weighting the layers differently on the style?
2. What happens if you weight the first few layers more heavily?
3. What happens if we weight the last layers more heavily?
4. What is the effect of the $\alpha$ parameter?
5. What is the effect of the $\beta$ parameter?
6. What do you think is a good configuration of the style layer weight parameters, $\alpha$ and $\beta$, and why?

### Entrez votre solution à Q3F dans la cellule ci-dessous

### Enter your answer to Q3F in the cell below.

1.pouvoir obtenir des informations nettes ou floutées sur les images

2. Avoir des informations détaillées sur les pixels du style

3. Avoir des informations de l'image floutée sur le contenu

4.$\alpha$ est le facteur de pondération pour la reconstruction du contenu

5.$\beta$ est le facteur de pondération pour la reconstruction du style

6. 